# Analysis of Nanopore sequencing data

## Introduction


This notebook describes how members of the Center for Antibody Technologies at DTU can analyse the sequences of their nanobody clones using nanopore sequening.




## 0: Setup

Copy data from the minion computer. Results are stored in Var/Lib/Minknow/Data.


## 1: Update variables
This should be the only code in the notebook you need to edit. Update this section so that the variables point to your data. The outdir should already exist and should contain the Guppy configuration files as described in the Guppy manual.

In [ ]:
# The location of the nanopore output fastq files
BASECALLS="/Users/cmaob/Documents/Nanopore/Analysis/Minion_raw_results/07062023_CAT_Run4/07062023_CAT_Run4/20230607_1102_MN37845_AOD586_2ba092dd/fastq_pass"

# The location where you want the analysis to be saved
OUTDIR="/Users/cmaob/Documents/Nanopore/Analysis"

# Name of this experiment
EXPNAME="07062023_CAT_Run4"

# File containing which clones are with what barcode
MAPPINGFILE="/Users/cmaob/Documents/Nanopore/Analysis/MA_yeswo_SVMP_001_mappings.csv"

# Location of Guppy executable
GUPPY="/Users/cmaob/Documents/Nanopore/ont-guppy-cpu/bin/guppy_barcoder"


## 2: Setup
Create the folders necessary for the analysis.

In [ ]:
%cd {OUTDIR}
%mkdir -p {EXPNAME}
%cd {EXPNAME}
%mkdir -p Consensus
%mkdir -p Demultiplex

## 3: Demultiplex the samples

Use the Guppy software from Nanopore Technologies to split the reads up according to their plate barcode.

### 3.1: Split the barcodes by plate

In [ ]:
guppy_plate_cmd = f"{GUPPY}\
    --input_path {BASECALLS} \
    --save_path Demultiplex \
    --data_path ../custom_barcoding_info_plate/barcoding \
    --barcode_kits MY-CUSTOM-BARCODES"

!{guppy_plate_cmd}

### Plotting the results - the numbers of reads per barcode

Enable R to be run in the notebook. All figures are plotted in R. This is indicated by the ```%%R``` at the beginning of the code cell. 

In [ ]:
%load_ext rpy2.ipython

Create bar charts of how many reads were assigned to each plate:

In [ ]:
%%R
library(ggplot2)
data <- read.table("Demultiplex/barcoding_summary.txt", header=TRUE, check.names=FALSE, sep="\t")
df <- as.data.frame(table(data$barcode_arrangement))
numb_labels <- round(df$Freq/sum(df$Freq) * 100, 1)
numb_labels <- paste(numb_labels, "%", sep="")

# Create the plot
p <- ggplot(data=df, aes(x=Var1, y=Freq)) +
  geom_bar(stat="identity", fill="steelblue", width = 0.75) +
  geom_text(aes(label=numb_labels), hjust=-0.2, color="black", size=3.5)+
  scale_x_discrete(limits=c("barcode04","barcode03","barcode02","barcode01","unclassified")) +
  scale_y_continuous(expand = c(0, 0)) +
  labs(y = "Read count", x="Barcode") +
  coord_flip(clip = "off") +
  theme_bw() +
  theme(panel.border = element_blank(), 
        axis.line = element_line(colour = "black"), 
        plot.margin = margin(0.5, 2, 0.5, 0.5, "cm"),
        axis.text= element_text(colour="black"),
        axis.title.x = element_text(vjust = -1)
  ) 

# Show the plot
print(p, vp=grid::viewport(width=unit(7, 'inch'), height=unit(5, 'inch')))
# Save the plot
ggsave("Demultiplex/Demultiplexing_by_plate.pdf", width = 7, height = 5, units = "in")
ggsave("Demultiplex/Demultiplexing_by_plate.png", width = 7, height = 5, units = "in")

### 3.2: Split the barcodes by well

Run Guppy again, but this time to split the reads by well barcode.

In [ ]:
for x in range(1, 5):
    guppy_well_cmd = f"{GUPPY} \
        --input_path Demultiplex/barcode{x:02d} \
        --save_path Demultiplex/barcode{x:02d}/demultiplex_by_well \
        --data_path ../custom_barcoding_info_well/barcoding \
        --barcode_kits MY-CUSTOM-BARCODES"

    !{guppy_well_cmd}

### Plotting the results - the numbers of reads per barcode

Now make a plot about the distribution of the barcodes.

The plots will be a little difficult to see in the panel of this notebook, but they should save to file okay. 

In [ ]:
%%R -w 5 -h 5 --units in -r 200

#Import data
bc1 <- read.table("Demultiplex/barcode01/demultiplex_by_well/barcoding_summary.txt", header=TRUE, check.names=FALSE, sep="\t")
bc2 <- read.table("Demultiplex/barcode02/demultiplex_by_well/barcoding_summary.txt", header=TRUE, check.names=FALSE, sep="\t")
bc3 <- read.table("Demultiplex/barcode03/demultiplex_by_well/barcoding_summary.txt", header=TRUE, check.names=FALSE, sep="\t")
bc4 <- read.table("Demultiplex/barcode04/demultiplex_by_well/barcoding_summary.txt", header=TRUE, check.names=FALSE, sep="\t")

# Turn into data frame
Barcode01 <- as.data.frame(table(bc1$barcode_arrangement))
Barcode02 <- as.data.frame(table(bc2$barcode_arrangement))
Barcode03 <- as.data.frame(table(bc3$barcode_arrangement))
Barcode04 <- as.data.frame(table(bc4$barcode_arrangement))

# Create graphing function
create_graph <- function(table) {
  ggplot(data=table, aes(x=Var1, y=Freq)) +
    geom_bar(stat="identity", fill="steelblue", width = 0.75) +
    scale_y_continuous(expand = c(0, 0)) +
    labs(y ="Read count", x=deparse(substitute(table))) +
    guides(x = guide_axis(angle = 90)) +
    theme_bw() +
    theme(panel.border = element_blank(), 
          axis.line = element_line(colour = "black"), 
          axis.text= element_text(colour="black", size = 8),
          axis.title.x = element_text(vjust = -0.75)
    )
}

# Call the function to create the graphs
p1 <- create_graph(Barcode01)
p2 <- create_graph(Barcode02)
p3 <- create_graph(Barcode03)
p4 <- create_graph(Barcode04)

# Stitch the plots together
library(patchwork)
well_plot <- (p1 / p2 / p3 / p4)
# Display the graphs
print(well_plot)
# Save the plot
ggsave("Demultiplex/Demultiplexing_by_well.pdf", width = 11, height = 9, units = "in")
ggsave("Demultiplex/Demultiplexing_by_well.png", width = 11, height = 9, units = "in")

Because there are so many unclassified reads it is difficult to see the numbers for the other barcodes. So we will generate a second graph without unclassified reads.

In [ ]:
%%R -w 5 -h 10 --units in -r 200

# Drop the unclassified row
Barcode01 <- subset(Barcode01,Var1!='unclassified' )
Barcode02 <- subset(Barcode02,Var1!='unclassified' )
Barcode03 <- subset(Barcode03,Var1!='unclassified' )
Barcode04 <- subset(Barcode04,Var1!='unclassified' )

# Create graphing function
create_graph <- function(table) {
  ggplot(data=table, aes(x=Var1, y=Freq)) +
    geom_bar(stat="identity", fill="steelblue", width = 0.75) +
    scale_y_continuous(expand = c(0, 0)) +
    labs(y ="Read count", x=deparse(substitute(table))) +
    #coord_flip(clip = "off") +
    guides(x = guide_axis(angle = 90)) +
    theme_bw() +
    theme(panel.border = element_blank(), 
          axis.line = element_line(colour = "black"), 
          #plot.margin = margin(0.5, 2, 0.5, 0.5, "cm"),
          axis.text= element_text(colour="black", size = 8),
          axis.title.x = element_text(vjust = -0.75)
    )
}

# Call the function to create the graphs
p1 <- create_graph(Barcode01)
p2 <- create_graph(Barcode02)
p3 <- create_graph(Barcode03)
p4 <- create_graph(Barcode04)

# Stitch the plots together
library(patchwork)
well_plot <- (p1 / p2 / p3 / p4)
# Display the graphs
print(well_plot)
# Save the plot
ggsave("Demultiplex/Demultiplexing_by_well_no_unclassified.pdf", width = 11, height = 9, units = "in")
ggsave("Demultiplex/Demultiplexing_by_well_no_unclassified.png", width = 11, height = 9, units = "in")

## 4: Create the consensus sequence

Run the medaka software to create consensus sequences on all 96 clones. This can take a while to run.



In [ ]:
# Get the name of the fastq file
DIR_PATH = "Demultiplex/barcode01/demultiplex_by_well/barcode96/"
FILENAMES = !ls "$DIR_PATH"
FILENAME = FILENAMES[0] if FILENAMES else None

# Iterate over filenames and run medaka_consensus
for i in range(1, 3):
    for j in range(1, 97):
        # Construct the medaka_consensus command
        medaka_consensus_cmd = f'medaka_consensus -i "Demultiplex/barcode{i:02d}/demultiplex_by_well/barcode{j:02d}/{FILENAME}" -d ../VHH_long_extended_to_primer.fa -o "Consensus/barcode{i:02d}_{j:02d}" -m r941_min_high_g303'
        # Execute the medaka_consensus command
        !{medaka_consensus_cmd}

Now add the name of the clone to the folder, filename and fasta header.

In [ ]:
%cd Consensus
# Create dictionary with the well locations of each clone name
dict = {}
with open(MAPPINGFILE) as f:
  for line in f:
      tok = line.strip().split("\t")
      dict[tok[0]] = tok[1]

In [ ]:
# Rename the folders
import os
from os.path import join
for oldname, newname in dict.items():
    fullname = newname + "_" + oldname
    os.rename(oldname, fullname)


In [ ]:
# Rename file to have the same name as the folder
for root,_, files in os.walk(os.getcwd()):
    for name in files:
        dir_name = os.path.basename(root)
        newname = dir_name + "_" + name
        os.rename(join(root,name),join(root,newname))

Next we need to move all fastas to a new folder to make them easier to work with

In [ ]:
# Create folder to store fastas
!mkdir all_fastas

# Copy the fastas into this folder
!find . -type f -iname "*.fasta" -exec cp -t all_fastas/ {} +

In [ ]:
# Rename the fasta header
%cd all_fastas
!mkdir renamed_headers

!for FILE in *.fasta; \
do awk '/^>/ {gsub(/.fa(sta)?$/,"",FILENAME);printf(">%s\n",FILENAME);next;} {print}' $FILE > renamed_headers/${FILE} ; \
done
# Source: https://www.biostars.org/p/204541/

In [ ]:
# Remove all files that don't begin with 'TPL' (or 'M_TPL' - edit below as needed)as they are not from clones
!find renamed_headers/ -type f ! -name "M_TPL*" -delete

In [ ]:
# Combine all fasta files into one multifasta
!cat renamed_headers/*.fasta > renamed_headers/nanopore_sequencing_result_${EXPNAME}.fasta
# Also create a copy in the experiment home folder so it is easy to find
!cp renamed_headers/nanopore_sequencing_result_${EXPNAME}.fasta ../../

## 5: Compare to Sanger Sequencing

Now that we have the sequence for each clone, we need to compare to the sanger sequences. To do this, we will use the tutorial that Yessica made. Materials for this can be found in the /Volumes/Bio-Temp/Center-for-Antibody-Technologies-temp/Wellcome/Research/Hand over from Yessica/CLC tutorials folder.